In [270]:
import pandas as pd
from datetime import datetime, timedelta
import requests 
import numpy as np
import mibian
import os
import math
from scipy.stats import norm

In [272]:
transfers = pd.read_csv('../data collection/blockchain logs/transfers.csv')
transfers = transfers[['fromAcc', 'toAcc', 'subId', 'amount', 'tradeId', 'expiry', 'strike', 'type', 'block_number']]

C:\Users\wnww71\AppData\Local\Temp\ipykernel_1540\3157368191.py:1: DtypeWarning: Columns (18,20) have mixed types. Specify dtype option on import or set low_memory=False.
  transfers = pd.read_csv(f'{final_file_name}.csv')


In [273]:
# Function to estimate the trade_date for a given block number
def estimate_date(block_number, highest_block, highest_block_time, interval):
    block_diff = highest_block - block_number
    estimated_time = highest_block_time - timedelta(seconds=int(block_diff * interval))
    return estimated_time.strftime('%Y-%m-%d')

def estimate_datetime(block_number, highest_block, highest_block_time, interval):
    block_diff = highest_block - block_number
    estimated_time = highest_block_time - timedelta(seconds=int(block_diff * interval))
    return estimated_time.strftime('%Y-%m-%d %H:%M:%S')
# # find the date that this instrument was traded on blockchain and query deribit data based on that
# # first we need to get time from block number
# # highest block number is trades_on_blockchain['block_number'][0] = 10687289, we get timestamp of this block
def add_date_columns(df):
    highest_block = df.block_number[0]
    response = requests.get(f"https://explorer.lyra.finance/api/v2/blocks/{highest_block}", headers={'accept': 'application/json'})
    first_timestamp = response.json()['timestamp']
    time_format = '%Y-%m-%dT%H:%M:%S.%fZ'
    parsed_first_time = datetime.strptime(first_timestamp, time_format)

    # iterate over block numbers and estimate their trade_date based on the block number difference from highest block
    # block production intervals are 2 seconds

    # # Iterate over block numbers and estimate their dates
    df['trade_date'] = df['block_number'].apply(
        lambda block_num: estimate_date(block_num, highest_block, parsed_first_time, 2)
    )

    df['trade_datetime'] = df['block_number'].apply(
        lambda block_num: estimate_datetime(block_num, highest_block, parsed_first_time, 2)
    )

    return df

transfers_wdate = add_date_columns(transfers.copy())
transfers_wdate.to_csv('../data collection/blockchain logs/transfers-wdate.csv')

In [274]:
def extract_traded_instruments(data):
    instruments = data[data['expiry'].notna()][['expiry', 'strike', 'type', 'trade_date', 'trade_datetime']]
    btc_instruments = instruments[instruments['strike'] > 10000]
    eth_instruments = instruments[instruments['strike'] <= 10000]
    return instruments, btc_instruments, eth_instruments

traded_instruments, traded_btc_instruments, traded_eth_instruments = extract_traded_instruments(transfers_wdate)

In [275]:
def add_instrument_name_column(instruments, ticker):
    for i, ins in instruments.iterrows():
        expiration_date = datetime.strptime(ins['expiry'], "%Y-%m-%d %H:%M:%S").strftime("%Y%m%d")
        strike_price =  int(ins['strike'])
        option_type = 'C' if ins['type'] == 'call' else 'P'
        formatted_string = f"{ticker}-{expiration_date}-{strike_price}-{option_type}"
        instruments.at[i, 'instrument'] = formatted_string

add_instrument_name_column(traded_eth_instruments, "ETH-USD")
add_instrument_name_column(traded_btc_instruments, "BTC-USD")

C:\Users\wnww71\AppData\Local\Temp\ipykernel_1540\390953901.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  instruments.at[i, 'instrument'] = formatted_string
C:\Users\wnww71\AppData\Local\Temp\ipykernel_1540\390953901.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  instruments.at[i, 'instrument'] = formatted_string


In [276]:
def extract_deribit_traded_instruments_from_ccdata():
    response = requests.get('https://data-api.ccdata.io/options/v1/markets/instruments',
        params={
            "market":"deribit",
            "instrument_status": ["ACTIVE", "IGNORED", "RETIRED", "EXPIRED"],
            "api_key":"37a2264c8cae2fc3b70b80f557ab6f559f50f86edd8a07f01a724edcf2512886"
        },
        headers={"Content-type":"application/json; charset=UTF-8"}
    )

    json_response = response.json()
    instruments = json_response['Data']['deribit']['instruments']
    # Create a DataFrame from the instruments data
    df = pd.DataFrame.from_dict(instruments, orient='index')
    return df

deribit_instruments = extract_deribit_traded_instruments_from_ccdata()
deribit_instruments.to_csv('../data collection/deribit instruments ohlcv/deribit_instruments.csv', index=True)

In [277]:
deribit_btc_instrument_names = deribit_instruments[deribit_instruments['MAPPED_INSTRUMENT'].str.startswith('BTC')]['MAPPED_INSTRUMENT']
deribit_eth_instrument_names = deribit_instruments[deribit_instruments['MAPPED_INSTRUMENT'].str.startswith('ETH')]['MAPPED_INSTRUMENT']

traded_btc_instrument_name = traded_btc_instruments['instrument'].unique()
traded_eth_instrument_name = traded_eth_instruments['instrument'].unique()

def get_matched_elements(names1, names2):
    matched_names = []
    for n1 in names1:
        for n2 in names2:
            if(n1 == n2):
                matched_names.append(n1)
    return set(matched_names)
            
# matched instruments between Lyra and Deribit
matched_btc_instruments = get_matched_elements(traded_btc_instrument_name, deribit_btc_instrument_names)
matched_eth_instruments = get_matched_elements(traded_eth_instrument_name, deribit_eth_instrument_names)

print(f'Out of {len(traded_btc_instrument_name)} btc traded instruments on Lyra, {len(matched_btc_instruments)} have been matched with instruments on Deribit.')
print(f'Out of {len(traded_eth_instrument_name)} eth traded instruments on Lyra, {len(matched_eth_instruments)} have been matched with instruments on Deribit.')

Out of 1103 btc traded instruments on Lyra, 682 have been matched with instruments on Deribit.
Out of 2565 eth traded instruments on Lyra, 1439 have been matched with instruments on Deribit.


In [278]:
# save historical ohlcv data for instruments, some of them are not available in Deribit
def save_ohlc_historical_data_for_instruments(instruments):
    for ins in instruments:
        asset = ins.split('-')[0].lower()
        file_path = f'../data collection/deribit instruments ohlcv/{asset}/{ins}.csv'
        # if file already exist skip
        if os.path.exists(file_path):
            print(f"{asset}/{ins} already exists.")
            break

        response = requests.get(
            'https://data-api.ccdata.io/options/v1/historical/days',
            params={
                "market": "deribit",
                "instrument": ins,
                "limit": 3000,
                "aggregate": 1,
                "fill": "false",
                "apply_mapping": "true",
                "response_format": "CSV",
                "api_key": "37a2264c8cae2fc3b70b80f557ab6f559f50f86edd8a07f01a724edcf2512886"
            },
            headers={"Content-type": "application/json; charset=UTF-8"}
        )

        if response.status_code == 200:            
            # Save the content to a CSV file
            with open(file_path, 'w') as file:
                file.write(response.text)
                print(f'{asset}/{ins} saved.')

save_ohlc_historical_data_for_instruments(matched_btc_instruments)
save_ohlc_historical_data_for_instruments(matched_eth_instruments)


btc/BTC-USD-20240927-70000-C already exists.
eth/ETH-USD-20240726-3000-P already exists.


In [279]:
# iterate over ohlcv directory files, for each instrument read their timestamp and extract the row
# that matches with estimated_timestamp of the same instrument in trades_on_blockchain

def merge_instruments_based_on_date(traded, asset):
    merged = pd.DataFrame()
    directory = f'../data collection/deribit instruments ohlcv/{asset}/'
    print(directory)
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        instrument_name = file_path.split('/')[4].split('.')[0]
        ohclv_data = pd.read_csv(file_path)
        ohclv_data['DATE'] = ohclv_data['TIMESTAMP'].apply(lambda ts:  datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d'))
        
        # Filter the trades with the specified instrument
        filtered_trades = traded[traded['instrument'] == instrument_name].copy()
        
        open_dict = ohclv_data.set_index('DATE')['OPEN'].to_dict()
        high_dict = ohclv_data.set_index('DATE')['HIGH'].to_dict()
        low_dict = ohclv_data.set_index('DATE')['LOW'].to_dict()
        close_dict = ohclv_data.set_index('DATE')['CLOSE'].to_dict()
        last_trade_price_dict = ohclv_data.set_index('DATE')['LAST_TRADE_PRICE'].to_dict()
        number_of_contracts_dict = ohclv_data.set_index('DATE')['NUMBER_OF_CONTRACTS'].to_dict()
        total_trades_dict = ohclv_data.set_index('DATE')['TOTAL_TRADES'].to_dict()
        total_trades_buy_dict = ohclv_data.set_index('DATE')['TOTAL_TRADES_BUY'].to_dict()
        total_trades_sell_dict = ohclv_data.set_index('DATE')['TOTAL_TRADES_SELL'].to_dict()
        volume_dict = ohclv_data.set_index('DATE')['VOLUME'].to_dict()
        
        filtered_trades['open'] = filtered_trades['trade_date'].map(open_dict)
        filtered_trades['high'] = filtered_trades['trade_date'].map(high_dict)
        filtered_trades['low'] = filtered_trades['trade_date'].map(low_dict)
        filtered_trades['close'] = filtered_trades['trade_date'].map(close_dict)
        filtered_trades['last_trade_price'] = filtered_trades['trade_date'].map(last_trade_price_dict)
        filtered_trades['number_of_contracts'] = filtered_trades['trade_date'].map(number_of_contracts_dict)
        filtered_trades['total_trades'] = filtered_trades['trade_date'].map(total_trades_dict)
        filtered_trades['total_trades_buy'] = filtered_trades['trade_date'].map(total_trades_buy_dict)
        filtered_trades['total_trades_sell'] = filtered_trades['trade_date'].map(total_trades_sell_dict)
        filtered_trades['volume'] = filtered_trades['trade_date'].map(volume_dict)


        merged = pd.concat([merged, filtered_trades],axis=0 ,ignore_index=True)
    
    return merged
    
merged_traded_and_deribit_btc_instruments = merge_instruments_based_on_date(traded_btc_instruments, 'btc').drop_duplicates(keep='first')
merged_traded_and_deribit_eth_instruments = merge_instruments_based_on_date(traded_eth_instruments, 'eth').drop_duplicates(keep='first')


../data collection/deribit instruments ohlcv/btc/
../data collection/deribit instruments ohlcv/eth/


In [280]:
# remove instruments without any Deribit data present for
merged_traded_and_deribit_btc_instruments = merged_traded_and_deribit_btc_instruments[merged_traded_and_deribit_btc_instruments['open'].notna()]
merged_traded_and_deribit_eth_instruments = merged_traded_and_deribit_eth_instruments[merged_traded_and_deribit_eth_instruments['open'].notna()]
merged_traded_and_deribit_instruments = pd.concat([merged_traded_and_deribit_btc_instruments, merged_traded_and_deribit_eth_instruments])

print('BTC traded instruments with data on both Lyra and Deribit: ', len(merged_traded_and_deribit_btc_instruments))
print('ETH traded instruments with data on both Lyra and Deribit:', len(merged_traded_and_deribit_eth_instruments))

BTC traded instruments with data on both Lyra and Deribit:  1182
ETH traded instruments with data on both Lyra and Deribit: 5657


In [281]:
def find_matching_pairs(group):
    trades = []
    for _, row1 in group.iterrows():
        for _, row2 in group.iterrows():
            if row1['fromAcc'] == row2['toAcc'] and row1['toAcc'] == row2['fromAcc'] and (row1['strike'] or  row2['strike']):
                trades.append((row1, row2))
    return trades

# find transfer pairs that shape an option trade together
def identify_transfer_pairs(trnsfrs):
    # Step 1: Identify tradeIds with at least one valid options trade
    valid_trade_ids = trnsfrs.dropna(subset=['strike', 'expiry'])['tradeId'].unique()

    # Step 2: Filter dataset for these tradeIds
    trade_with_option_involved = trnsfrs[trnsfrs['tradeId'].isin(valid_trade_ids)]
    
    # Step 3: Find matching pairs of trades where fromAcc and toAcc are reversed
    return trade_with_option_involved.groupby('tradeId').apply(find_matching_pairs)

pairs = identify_transfer_pairs(transfers_wdate)

In [282]:
def create_option_trades_from_transfer_pairs(pairs):
    structured_trades = []
    for transfer_pairs in pairs:
        for transfer1, transfer2 in transfer_pairs:        
            ticker = 'ETH-USD'
            seller = transfer2 if math.isnan(transfer1['strike']) else transfer1
            buyer = transfer1 if math.isnan(transfer1['strike']) else transfer2
            if(seller['strike'] > 20000):
                ticker = 'BTC-USD'
            expiration_date = pd.to_datetime(seller['expiry']).strftime("%Y%m%d")
            strike_price =  int(seller['strike'])
            option_type = 'C' if seller['type'] == 'call' else 'P'
            new_row = {
                'tradeId': seller['tradeId'],
                'instrument': f"{ticker}-{expiration_date}-{strike_price}-{option_type}",
                'trade_date': buyer['trade_date'],
                'trade_datetime': buyer['trade_datetime'],
                'premium': int(buyer['amount']) / 1e18,
                'amount': int(seller['amount']) / 1e18,
                'price': int(buyer['amount']) / int(seller['amount']),
                'acc': buyer['fromAcc'],
                'acc2': buyer['toAcc'],
            }
            structured_trades.append(new_row)
    return structured_trades



option_trades = pd.DataFrame(create_option_trades_from_transfer_pairs(pairs))
option_trades.to_csv('./deribit lyra comparison/option_trades_on_lyra.csv')
option_trades.head()

,tradeId,instrument,trade_date,trade_datetime,premium,amount,price,acc,acc2
0,315199,ETH-USD-20240416-3400-C,2024-04-14,2024-04-14 10:42:07,-108.5,-5.0,21.7,585,166
1,315199,ETH-USD-20240416-3400-C,2024-04-14,2024-04-14 10:42:07,-108.5,-5.0,21.7,585,166
2,315200,ETH-USD-20240416-2700-P,2024-04-14,2024-04-14 10:44:19,-68.5,-5.0,13.7,585,166
3,315200,ETH-USD-20240416-2700-P,2024-04-14,2024-04-14 10:44:19,-68.5,-5.0,13.7,585,166
4,315207,ETH-USD-20240416-3400-C,2024-04-14,2024-04-14 10:49:47,5.0,5.0,1.0,330,166


In [283]:
def merge_on_instrument_and_trade_date(left, right):
    right['instrument_date_key'] = right['trade_date'] + ' ' + right['instrument']
    left['instrument_date_key'] = left['trade_date'] + ' ' + left['instrument']
    merged_data = pd.merge(left, right, on='instrument_date_key', how='left')
    merged_data = merged_data[merged_data['open'].notna()]
    merged_data = merged_data[merged_data['expiry'].notna()]
    merged_data = merged_data[merged_data['price'] != 1.0]
    return merged_data.drop_duplicates(keep='first')

merged_data_of_lyra_trades_and_deribit_ohlcv = merge_on_instrument_and_trade_date(option_trades, merged_traded_and_deribit_instruments)

In [284]:
def get_spot_historical_price(asset):
    response = requests.get('https://data-api.cryptocompare.com/spot/v1/historical/days',
        params={"instrument":f"{asset}-USDC","limit":3000,"aggregate":1,"fill":"true","apply_mapping":"true","response_format":"JSON","market":"deribit","api_key":"37a2264c8cae2fc3b70b80f557ab6f559f50f86edd8a07f01a724edcf2512886"},
        headers={"Content-type":"application/json; charset=UTF-8"}
    )

    json_response = response.json()
    data = pd.DataFrame(json_response['Data'])
    data.to_csv(f'./market data/{asset}-spot.csv')
    return data

def add_spot_data_based_on_date(trade_data, spot_data):
    spot_data['trade_date_x'] = spot_data['TIMESTAMP'].apply(lambda d: datetime.fromtimestamp(d).strftime("%Y-%m-%d"))
    trade_data = trade_data.merge(spot_data[['trade_date_x', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOLUME']], on='trade_date_x')
    trade_data['open_in_usd'] = trade_data['OPEN'] * trade_data['open']
    trade_data['high_in_usd'] = trade_data['HIGH'] * trade_data['high']
    trade_data['low_in_usd'] = trade_data['LOW'] * trade_data['low']
    trade_data['close_in_usd'] = trade_data['CLOSE'] * trade_data['close']
    trade_data['volume_in_usd'] = trade_data['VOLUME'] * trade_data['volume']
    return trade_data

btc_merged_data_of_lyra_trades_and_deribit_ohlcv = add_spot_data_based_on_date(
    merged_data_of_lyra_trades_and_deribit_ohlcv[merged_data_of_lyra_trades_and_deribit_ohlcv['instrument_x'].str.startswith('BTC')],
    get_spot_historical_price('BTC')
)


eth_merged_data_of_lyra_trades_and_deribit_ohlcv = add_spot_data_based_on_date(
    merged_data_of_lyra_trades_and_deribit_ohlcv[merged_data_of_lyra_trades_and_deribit_ohlcv['instrument_x'].str.startswith('ETH')],
    get_spot_historical_price('ETH'),
)

In [285]:
def batch_deribit_prices(data):
    data['deribit_contracts_ohlc_prices'] = data.apply(lambda row: (row['open_in_usd'], row['high_in_usd'],	row['low_in_usd'], row['close_in_usd']), axis=1)
    data['deribit_spot_ohlc_prices'] = data.apply(lambda row: (row['OPEN'], row['HIGH'], row['LOW'], row['CLOSE']), axis=1)
    return data[['deribit_contracts_ohlc_prices',  'deribit_spot_ohlc_prices', 'price', 'expiry', 'strike', 'type', 'trade_date_x', 'trade_datetime_x']]

btc_merged_data_of_lyra_trades_and_deribit_ohlcv = batch_deribit_prices(btc_merged_data_of_lyra_trades_and_deribit_ohlcv)
eth_merged_data_of_lyra_trades_and_deribit_ohlcv = batch_deribit_prices(eth_merged_data_of_lyra_trades_and_deribit_ohlcv)

In [286]:
def get_interest_rates():
    interest_rates = pd.read_csv('./market data/stake-rates.csv')
    interest_rates['trade_date_x'] = [datetime.strptime(date, "%Y-%m-%dT%H:%M:%S").strftime("%Y-%m-%d") for date in interest_rates['Date']]
    return interest_rates

def merge_based_on_trade_date(left, right):
    return pd.merge(left, right, on='trade_date_x', how='left')

btc_merged_data_of_lyra_trades_and_deribit_ohlcv = merge_based_on_trade_date(
    btc_merged_data_of_lyra_trades_and_deribit_ohlcv, 
    get_interest_rates()
)

eth_merged_data_of_lyra_trades_and_deribit_ohlcv = merge_based_on_trade_date(
    eth_merged_data_of_lyra_trades_and_deribit_ohlcv, 
    get_interest_rates()
)

In [287]:
def rename_traded_and_lyra_prices_column(data):
    data.columns = ['deribit_ohlc_prices', 'deribit_spot_prices', 'lyra_price', 'expiry', 'strike', 'type', 'date', 'datetime', 'sdfsdf', 'interest_rate']
    return data.drop(['sdfsdf'], axis=1)

eth_merged_data_of_lyra_trades_and_deribit_ohlcv = rename_traded_and_lyra_prices_column(eth_merged_data_of_lyra_trades_and_deribit_ohlcv)
btc_merged_data_of_lyra_trades_and_deribit_ohlcv = rename_traded_and_lyra_prices_column(btc_merged_data_of_lyra_trades_and_deribit_ohlcv)

In [288]:
def calculate_fraction_of_year(start_time, end_time, fmt2='%Y-%m-%d %H:%M:%S', fmt1='%Y-%m-%d %H:%M:%S'):
    # Parse the datetime strings
    start_time = datetime.strptime(start_time, fmt1)
    end_time = datetime.strptime(end_time, fmt2)
    
    # Calculate the difference in seconds
    delta_seconds = (end_time - start_time).total_seconds()
    
    # Total seconds in a year (assuming 365.25 days per year for leap years)
    seconds_in_year = 365.25 * 24 * 3600
    
    # Calculate the fraction of the year
    return delta_seconds / seconds_in_year 

eth_merged_data_of_lyra_trades_and_deribit_ohlcv['time_to_maturity'] = eth_merged_data_of_lyra_trades_and_deribit_ohlcv.apply(lambda row: calculate_fraction_of_year(row['datetime'], row['expiry']), axis=1)
btc_merged_data_of_lyra_trades_and_deribit_ohlcv['time_to_maturity'] = btc_merged_data_of_lyra_trades_and_deribit_ohlcv.apply(lambda row: calculate_fraction_of_year(row['datetime'], row['expiry']), axis=1)


In [289]:
def calculate_deribit_ohlc_IVs_from_mibian(row):
    # Assuming the underlying asset price 'S' is needed and provided in an additional column
    # and that the interest rate 'r' is provided in percentage terms
    K = row['strike']  # Strike price
    r = row['interest_rate']  # Annual interest rate
    T = row['time_to_maturity']  # Time to maturity in years
    flag = 'c' if row['type'] == 'call' else 'p'

    # Calculating open IV for Deribit price
    open = mibian.BS([row['deribit_spot_prices'][0], K, r, T*365], putPrice=row['deribit_ohlc_prices'][0] if flag == 'p' else None, callPrice=row['deribit_ohlc_prices'][0] if flag == 'c' else None)
    # Calculating high IV for Deribit price
    high = mibian.BS([row['deribit_spot_prices'][1], K, r, T*365], putPrice=row['deribit_ohlc_prices'][1] if flag == 'p' else None, callPrice=row['deribit_ohlc_prices'][1] if flag == 'c' else None)
    # Calculating low IV for Deribit price
    low = mibian.BS([row['deribit_spot_prices'][2], K, r, T*365], putPrice=row['deribit_ohlc_prices'][2] if flag == 'p' else None, callPrice=row['deribit_ohlc_prices'][2] if flag == 'c' else None)
    # Calculating close IV for Deribit price
    close = mibian.BS([row['deribit_spot_prices'][3], K, r, T*365], putPrice=row['deribit_ohlc_prices'][3] if flag == 'p' else None, callPrice=row['deribit_ohlc_prices'][3] if flag == 'c' else None)
    if(row.name%50 == 0):
        print(f'ohlcv {row.name} done')    
    return (open.impliedVolatility, high.impliedVolatility, low.impliedVolatility, close.impliedVolatility)


def select_spot_price_based_on_time(datetime, deribit_spot_prices):
    date_time_obj = pd.to_datetime(datetime, format='%Y-%m-%d %H:%M:%S')
    
    # Check if time is before or after noon (12:00)
    if date_time_obj.time() < pd.Timestamp("12:00").time():
        return deribit_spot_prices[0] # return open price
    else:
        return deribit_spot_prices[3] # return close price

def calculate_lyra_IV_from_mibian(row):
    # Assuming the underlying asset price 'S' is needed and provided in an additional column
    # and that the interest rate 'r' is provided in percentage terms

    # decide use either close or open spot price
    S = select_spot_price_based_on_time(row['datetime'] ,row['deribit_spot_prices'])
    K = row['strike']  # Strike price
    r = row['interest_rate']  # Annual interest rate
    T = row['time_to_maturity']  # Time to maturity in years
    flag = 'c' if row['type'] == 'call' else 'p'

    # # Calculating IV for Lyra price
    lyra = mibian.BS([S, K, r, T*365], putPrice=abs(row['lyra_price']) if flag == 'p' else None, callPrice=abs(row['lyra_price']) if flag == 'c' else None)
    if(row.name%50 == 0):
        print(f'Lyra {row.name} done')
    return lyra.impliedVolatility

btc_merged_data_of_lyra_trades_and_deribit_ohlcv['lyra_IV'] = btc_merged_data_of_lyra_trades_and_deribit_ohlcv.apply(lambda row: calculate_lyra_IV_from_mibian(row) , axis=1)
btc_merged_data_of_lyra_trades_and_deribit_ohlcv['deribit_ohlc_IVs'] = btc_merged_data_of_lyra_trades_and_deribit_ohlcv.apply(lambda row: calculate_deribit_ohlc_IVs_from_mibian(row) , axis=1)

eth_merged_data_of_lyra_trades_and_deribit_ohlcv['deribit_ohlc_IVs'] = eth_merged_data_of_lyra_trades_and_deribit_ohlcv.apply(lambda row: calculate_deribit_ohlc_IVs_from_mibian(row) , axis=1)
eth_merged_data_of_lyra_trades_and_deribit_ohlcv['lyra_IV'] = eth_merged_data_of_lyra_trades_and_deribit_ohlcv.apply(lambda row: calculate_lyra_IV_from_mibian(row) , axis=1)

Lyra 0 done
Lyra 50 done
Lyra 100 done
Lyra 150 done
Lyra 200 done
Lyra 250 done
Lyra 300 done
Lyra 350 done
Lyra 400 done
Lyra 450 done
Lyra 500 done
Lyra 550 done
Lyra 600 done
Lyra 650 done
Lyra 700 done
Lyra 750 done
Lyra 800 done
Lyra 850 done
Lyra 900 done
Lyra 950 done
Lyra 1000 done
Lyra 1050 done
Lyra 1100 done
Lyra 1150 done
Lyra 1200 done
Lyra 1250 done
Lyra 1300 done
Lyra 1350 done
Lyra 1400 done
Lyra 1450 done
Lyra 1500 done
Lyra 1550 done
Lyra 1600 done
Lyra 1650 done
Lyra 1700 done
Lyra 1750 done
Lyra 1800 done
Lyra 1850 done
Lyra 1900 done
Lyra 1950 done
Lyra 2000 done
Lyra 2050 done
Lyra 2100 done
Lyra 2150 done
Lyra 2200 done
Lyra 2250 done
ohlcv 0 done
ohlcv 50 done
ohlcv 100 done
ohlcv 150 done
ohlcv 200 done
ohlcv 250 done
ohlcv 300 done
ohlcv 350 done
ohlcv 400 done
ohlcv 450 done
ohlcv 500 done
ohlcv 550 done
ohlcv 600 done
ohlcv 650 done
ohlcv 700 done
ohlcv 750 done
ohlcv 800 done
ohlcv 850 done
ohlcv 900 done
ohlcv 950 done
ohlcv 1000 done
ohlcv 1050 done
ohlc

In [ ]:
# Define a function to round tuples
def round_tuple(t):
    return tuple(round(num, 2) for num in t)

def round_columns(data):
    # Apply this function to the Prices column
    data['deribit_ohlc_IVs'] = data['deribit_ohlc_IVs'].apply(round_tuple)
    data['deribit_spot_prices'] = data['deribit_spot_prices'].apply(round_tuple)
    data['deribit_ohlc_prices'] = data['deribit_ohlc_prices'].apply(round_tuple)
    data['lyra_price'] = data['lyra_price'].apply(lambda x: round(x, 2))
    data['lyra_IV'] = data['lyra_IV'].apply(lambda x: round(x, 2))
    data['selected_spot_price'] = data.apply(lambda row: select_spot_price_based_on_time(row['datetime'] ,row['deribit_spot_prices']), axis=1)
    return data

btc_merged_data_of_lyra_trades_and_deribit_ohlcv = round_columns(btc_merged_data_of_lyra_trades_and_deribit_ohlcv)
eth_merged_data_of_lyra_trades_and_deribit_ohlcv = round_columns(eth_merged_data_of_lyra_trades_and_deribit_ohlcv)

In [ ]:
eth_merged_data_of_lyra_trades_and_deribit_ohlcv.to_csv('./deribit lyra comparison/eth.csv')
btc_merged_data_of_lyra_trades_and_deribit_ohlcv.to_csv('./deribit lyra comparison/btc.csv')